In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from IPython.display import Markdown, display

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import cluster, datasets
from sklearn.cluster import KMeans

import random
from IPython.display import Markdown, display 


In [2]:
films_w_awards = pd.read_csv('films_w_awards.csv')


/var/folders/0r/l856hq9x61zf5p8g8vxjs_dh0000gn/T/ipykernel_1996/2686176247.py:1: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  films_w_awards = pd.read_csv('films_w_awards.csv')


In [3]:
# Use the 'sample' function to randomly sample 10% of the rows
sampled_films = films_w_awards.sample(frac=0.1, random_state=42)  # Setting random_state for reproducibility

# 'sampled_df' now contains 10% of the rows from 'all_films'
sampled_films.shape

(21357, 57)

In [4]:
# extracting categoricals
all_categorical = sampled_films.select_dtypes(np.object)

display(type(all_categorical),all_categorical.head(3), all_categorical.shape, all_categorical.columns)


/var/folders/0r/l856hq9x61zf5p8g8vxjs_dh0000gn/T/ipykernel_1996/101719188.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all_categorical = sampled_films.select_dtypes(np.object)


pandas.core.frame.DataFrame

,movie_name,director,actor1,actor2,actor3,actor4,genre1,genre2,genre3,osc_nominated_category1,...,osc_won_category10,osc_won_category11,osc_won_category12,cannes_won_category1,cannes_won_category2,cannes_won_category3,movie_id,director_id,star_id,description
35604,Cameron Decker: The Thin Blue Line,NaN,NaN,NaN,NaN,NaN,Action,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,tt10994398,NaN,NaN,Government contractor hired to go after a dome...
83004,Na yasnyy ogon,Vitaliy Koltsov,Tatyana Doronina,Lev Durov,Boris Khimichev,Aleksandr Lazarev,Adventure,War,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,tt0073433,/name/nm0464437/,"/name/nm0233985/,/name/nm0244296/,/name/nm0451...",NaN
27131,Con Girls,NaN,NaN,NaN,NaN,NaN,Action,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,tt6192784,NaN,NaN,"IN THE CITY OF ANGELS, THERE ARE 8 MILLION CON..."


(21357, 50)

Index(['movie_name', 'director', 'actor1', 'actor2', 'actor3', 'actor4',
       'genre1', 'genre2', 'genre3', 'osc_nominated_category1',
       'osc_nominated_category2', 'osc_nominated_category3',
       'osc_nominated_category4', 'osc_nominated_category5',
       'osc_nominated_category6', 'osc_nominated_category7',
       'osc_nominated_category8', 'osc_nominated_category9',
       'osc_nominated_category10', 'osc_nominated_category11',
       'osc_nominated_category12', 'osc_nominated_category13',
       'osc_nominated_category14', 'osc_nominated_category15',
       'osc_nominated_category16', 'osc_nominated_category17',
       'osc_nominated_category18', 'osc_nominated_category19',
       'osc_nominated_category20', 'osc_nominated_category21',
       'osc_nominated_category22', 'osc_won_category1', 'osc_won_category2',
       'osc_won_category3', 'osc_won_category4', 'osc_won_category5',
       'osc_won_category6', 'osc_won_category7', 'osc_won_category8',
       'osc_won_category

In [11]:
# Replace all NaN values with 0
categorical.fillna(0, inplace=True)
categorical

/var/folders/0r/l856hq9x61zf5p8g8vxjs_dh0000gn/T/ipykernel_1996/2948121644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical.fillna(0, inplace=True)


,director,actor1,actor2,actor3,actor4,genre1,genre2,genre3
35604,0,0,0,0,0,Action,0,0
83004,Vitaliy Koltsov,Tatyana Doronina,Lev Durov,Boris Khimichev,Aleksandr Lazarev,Adventure,War,0
27131,0,0,0,0,0,Action,0,0
146981,Sean Cain,Ezra Buzzington,Jason Mewes,Lauren Walsh,Jack Forcinito,Drama,Horror,Romance
104899,Amos Gitai,Dominique Sanda,Sapho,Elinor Jagodnik,Raphael Hatishbi,Fantasy,Mystery,0
...,...,...,...,...,...,...,...,...
174295,Christina Kallas,Bex,Vandit Bhatt,John J. Concado,Robert Z. Grant,Comedy,Drama,Family
201391,Philippe Martinez,Nathalie Cox,Armand Assante,Richard Carter,Jonathan Failla,Action,0,0
183958,Zdenko Molnar,Pavel Gyurosii,Kristian Ilka,Zdenko Molnar,Pavel Gyurosíi,Action,Adventure,0
16006,Jolie Kinga,Lucian Maisel,Mor Cohen,Rebecca Rapoport-Cole,Mark Perrone,Mystery,Thriller,0


In [12]:
# I choose the ones I'll use for clustering

categorical = all_categorical[['genre1', 'genre2', 'genre3']]



In [13]:
import pandas as pd

# Assuming 'categorical' is your DataFrame with categorical columns

# Get the list of categorical column names
cat_cols = categorical.columns.tolist()

# Perform one-hot encoding on the categorical columns
categorical_encoded = pd.get_dummies(categorical, columns=cat_cols)

# Rename the one-hot encoded columns with the original column names

# Print the resulting DataFrame
print(categorical_encoded)


        genre1_Action  genre1_Adult  genre1_Adventure  genre1_Animation  \
35604               1             0                 0                 0   
83004               0             0                 1                 0   
27131               1             0                 0                 0   
146981              0             0                 0                 0   
104899              0             0                 0                 0   
...               ...           ...               ...               ...   
174295              0             0                 0                 0   
201391              1             0                 0                 0   
183958              1             0                 0                 0   
16006               0             0                 0                 0   
124874              0             0                 1                 0   

        genre1_Biography  genre1_Comedy  genre1_Crime  genre1_Drama  \
35604                  0    

In [14]:
categorical_encoded.shape

(21357, 69)